In [215]:
import json
import numpy as np
import pandas as pd
import lxml

# Import SAP data previously mapped for ZA1

15Dec_D365_orders_ZA1.csv

In [216]:
df5 = pd.read_csv("./data/ZA1/15Dec_D365_orders_ZA1.csv", low_memory=False)

# Import Products, Customers and Warehouses for the Legal Entity

In [217]:
LE = input('Enter the legal entity company code')
# NA1, MZ1, UG1

In [218]:
if LE == 'NA1':
    LE_Products = pd.read_csv("./data/NA1/Group1Products_NA1.csv", delimiter="\t")

if LE == 'MZ1':
    LE_Products = pd.read_csv("./data/MZ1/Group1Products_MZ1.csv", delimiter=",")

if LE == 'UG1':
    LE_Products = pd.read_csv("./data/UG1/Group1Products_UG1.csv", delimiter=",")
    

In [219]:
LE_Products

,D365_ItemNo_LE,Material_Description_LE
0,5141,SCH NOVIDA PINEAPPLE 500ML 12 S/W NP
1,10208,SCH NOVIDA P/APPLE ZERO 350ML 12 S/W NP
2,1271,COKE 500ML 12 S/W NP
3,2062,SPRITE 500ML 20 RB
4,5157,SCH NOVIDA PINEAPPLE BREEZ 300ML 24 RB
...,...,...
76,5465,SCH TONIC 300ML 24 RB
77,7477,RWENZORI WATER 500ML (24 BOX) NP
78,11165,SPRITE ZERO 330ML 12 S/W NP
79,10811,MM REFRESH MANGO 280ML 12 S/W


In [220]:
if LE == 'NA1':
    LE_Customers = pd.read_csv("./data/NA1/NA1_Customers.csv", delimiter="\t")
    LE_Customers.rename(columns={'Validated Customer Accounts': 'D365_Cust_No_LE'}, inplace=True)
 

if LE == 'MZ1':
    LE_Customers = pd.read_csv("./data/MZ1/PERF01_MZ1_Customers.csv", delimiter="\t")    
    LE_Customers.rename(columns={'CustomerAccount': 'D365_Cust_No_LE'}, inplace=True)
    
if LE == 'UG1':
    LE_Customers = pd.read_csv("./data/UG1/IPT3 UG1- Customers.csv", delimiter=",")    
    LE_Customers.rename(columns={'CustomerAccount': 'D365_Cust_No_LE'}, inplace=True)
    LE_Customers = LE_Customers[['D365_Cust_No_LE']].copy()
    # Name


In [221]:
# Read list of direct customers from CE, and merge that with the F&O customers per legal entity

#CE_direct_customers = pd.read_csv('./data/Group1Customers.csv', delimiter=";")
CE_direct_customers = pd.read_csv('./data/customers/Group1Customers.csv', delimiter="\t")
CE_direct_customers.drop(columns={'name', 'ccba_cicorderplacementrulename'}, inplace=True, axis=1)
CE_direct_customers.rename(columns={'accountnumber': 'D365_Cust_No_LE'}, inplace=True)
CE_direct_customers = CE_direct_customers[CE_direct_customers['msdyn_companyname'] == LE].copy()
CE_direct_customers.drop(columns={'msdyn_companyname'}, inplace=True, axis=1)

In [222]:
# Merge based on Index
LE_Customers = pd.merge(
    LE_Customers,
    CE_direct_customers,
    how="inner",
    on="D365_Cust_No_LE",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [223]:
if LE == 'NA1':
    LE_Warehouses = pd.read_csv("./data/NA1/NA1_Validated  IPT3 Warehouse Export  NA1 Warehouses.csv", delimiter=",")
    LE_Warehouses.rename(columns={'D365_Del_Location': 'D365_Del_Loc_LE'}, inplace=True)
    LE_Warehouses = LE_Warehouses[['D365_Del_Loc_LE']].copy()
    #D365_Del_Location

if LE == 'MZ1':
    LE_Warehouses = pd.read_csv("./data/MZ1/IPT3 MZ1 PERF01 Warehouse.csv", delimiter=",")
    LE_Warehouses.rename(columns={'D365DELIVERYLOCATION': 'D365_Del_Loc_LE'}, inplace=True)
    LE_Warehouses = LE_Warehouses[['D365_Del_Loc_LE']].copy()
    #D365DELIVERYLOCATION

if LE == 'UG1':
    LE_Warehouses = pd.read_csv("./data/UG1/IPT3 WMS UG1-Warehouses.csv", delimiter=",")    
    LE_Warehouses.rename(columns={'D365_Del_Loc': 'D365_Del_Loc_LE'}, inplace=True)
    LE_Warehouses = LE_Warehouses[['D365_Del_Loc_LE']].copy()
    #D365_Del_Loc

In [224]:
LE_Warehouses

,D365_Del_Loc_LE
0,UG1-UG010B
1,UG1-UG011B
2,UG1-UG012B


## Get the top n warehouses and filter the dataframe for those

In [225]:
# Group by site_id and count unique Order Numbers
#warehouse_order_counts = df5.groupby('site_id')['Order Number'].nunique().reset_index(name='OrderLinesCount')
warehouse_order_counts = df5.groupby('D365_Del_Loc')['Order Number'].nunique().reset_index(name='OrderLinesCount')
#D365_Del_Loc 

# Sort by OrderLinesCount in descending order and get the top 9 warehouses
#top_warehouses = warehouse_order_counts.sort_values(by='OrderLinesCount', ascending=False).head(len(LE_Warehouses))
top_warehouses = warehouse_order_counts.sort_values(by='OrderLinesCount', ascending=False)

#top_warehouses['Grouping'] = (top_warehouses.index % 7) + 1


In [226]:
top_warehouses.reset_index(inplace=True, drop=True)
top_warehouses['Grouping'] = (top_warehouses.index % len(LE_Warehouses))

In [227]:
top_warehouses

,D365_Del_Loc,OrderLinesCount,Grouping
0,ZA1-ZA014B,1113,0
1,ZA1-ZA016G,983,1
2,ZA1-ZA516B,951,2
3,ZA1-ZA020B,922,0
4,ZA1-ZA032B,921,1
...,...,...,...
82,ZA1-ZA034B,16,1
83,ZA1-ZA200B,13,2
84,ZA1-ZA565B,10,0
85,ZA1-ZA039B,5,1


In [228]:
# Filter the original DataFrame to only include rows where site_id is in the list of top warehouses
#top_warehouse_ids = top_warehouses['D365_Del_Loc'].tolist()
#df5_W = df5[df5['D365_Del_Loc'].isin(top_warehouse_ids)]
df5_W = df5.copy()

In [229]:
top_warehouses.reset_index(inplace=True, drop=True)

## Get the top n customers and filter the dataframe for those

In [230]:
# Group by D365_Cust_No and count unique Order Numbers
customer_order_counts = df5_W.groupby('D365_Cust_No')['Order Number'].nunique().reset_index(name='OrderLinesCount')

# Sort by OrderLinesCount in descending order and get the top n customers
#top_customers = customer_order_counts.sort_values(by='OrderLinesCount', ascending=False).head(len(LE_Customers))
top_customers = customer_order_counts.sort_values(by='OrderLinesCount', ascending=False)

In [231]:
top_customers.reset_index(inplace=True, drop=True)

In [232]:
top_customers.reset_index(inplace=True, drop=True)
top_customers['Grouping'] = (top_customers.index % len(LE_Customers))

In [233]:
top_customers

,D365_Cust_No,OrderLinesCount,Grouping
0,10017885,73,0
1,10004521,63,1
2,10017888,57,2
3,10004547,56,3
4,10017883,47,4
...,...,...,...
7017,10021202,1,21
7018,10053913,1,0
7019,10054011,1,1
7020,10019883,1,2


In [234]:
#top_customer_ids = top_customers['D365_Cust_No'].tolist()
#df5_W_C = df5_W[df5_W['D365_Cust_No'].isin(top_customer_ids)]
df5_W_C = df5_W.copy()

## Get the top n products and filter the dataframe for those

In [235]:
product_order_counts = df5_W_C.groupby('D365_ItemNo')['Order Number'].nunique().reset_index(name='OrderLinesCount')


top_products = product_order_counts.sort_values(by='OrderLinesCount', ascending=False).head(len(LE_Products))


print(top_products)

    D365_ItemNo  OrderLinesCount
126        1040            12704
310        7579            10761
267        6540            10031
100       10207             9863
227        5740             9383
..          ...              ...
258        6366             2251
173        2518             2233
311        7584             2209
85        10173             2166
341        7988             2165

[81 rows x 2 columns]


In [236]:
top_product_ids = top_products['D365_ItemNo'].tolist()
df5_W_C_P = df5_W_C[df5_W_C['D365_ItemNo'].isin(top_product_ids)]

In [237]:
top_products.reset_index(inplace=True, drop=True)

## Now replace warehouses, customers and products for the data in the new legal entity
Extend the dataframes produced above, top_warehouses, top_customers and top_products by adding a column (for the same number of rows).  The new column contains the corresponding data for the new legal entity.

### Create Products lookup table

In [238]:
# Create a new column 'Index' in both dataframes with current index values
top_products['Index'] = top_products.index
LE_Products['Index'] = LE_Products.index

In [239]:
if len(LE_Products) != len(top_products):
    raise SystemExit("Mismatch in number of products")

In [240]:
LE_Products

,D365_ItemNo_LE,Material_Description_LE,Index
0,5141,SCH NOVIDA PINEAPPLE 500ML 12 S/W NP,0
1,10208,SCH NOVIDA P/APPLE ZERO 350ML 12 S/W NP,1
2,1271,COKE 500ML 12 S/W NP,2
3,2062,SPRITE 500ML 20 RB,3
4,5157,SCH NOVIDA PINEAPPLE BREEZ 300ML 24 RB,4
...,...,...,...
76,5465,SCH TONIC 300ML 24 RB,76
77,7477,RWENZORI WATER 500ML (24 BOX) NP,77
78,11165,SPRITE ZERO 330ML 12 S/W NP,78
79,10811,MM REFRESH MANGO 280ML 12 S/W,79


In [241]:
# Merge based on Index
top_products = pd.merge(
    top_products,
    LE_Products,
    how="left",
    on="Index",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [242]:
top_products.drop(columns={'OrderLinesCount', 'Index'}, inplace=True, axis=1)

In [243]:
top_products

,D365_ItemNo,D365_ItemNo_LE,Material_Description_LE
0,1040,5141,SCH NOVIDA PINEAPPLE 500ML 12 S/W NP
1,7579,10208,SCH NOVIDA P/APPLE ZERO 350ML 12 S/W NP
2,6540,1271,COKE 500ML 12 S/W NP
3,10207,2062,SPRITE 500ML 20 RB
4,5740,5157,SCH NOVIDA PINEAPPLE BREEZ 300ML 24 RB
...,...,...,...
76,6366,5465,SCH TONIC 300ML 24 RB
77,2518,7477,RWENZORI WATER 500ML (24 BOX) NP
78,7584,11165,SPRITE ZERO 330ML 12 S/W NP
79,10173,10811,MM REFRESH MANGO 280ML 12 S/W


### Create Customers lookup table

In [244]:
# Create a new column 'Index' in both dataframes with current index values
#top_customers['Index'] = top_customers.index
LE_Customers['Index'] = LE_Customers.index

In [245]:
top_customers.rename(columns={'Grouping': 'Index'}, inplace=True)

In [246]:
#if len(LE_Customers) != len(top_customers):
#    raise SystemExit("Mismatch in number of customers")

In [247]:
#LE_Customers = LE_Customers.drop_duplicates().copy()

In [248]:
# Merge based on Index
top_customers = pd.merge(
    top_customers,
    LE_Customers,
    how="left",
    on="Index",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [249]:
top_customers.drop(columns={'OrderLinesCount', 'Index'}, inplace=True, axis=1)

### Create Warehouses lookup table

In [250]:
# Create a new column 'Index' in both dataframes with current index values
#top_warehouses['Index'] = top_warehouses.index
LE_Warehouses['Index'] = LE_Warehouses.index

In [251]:
top_warehouses.rename(columns={'Grouping': 'Index'}, inplace=True)

In [252]:
#if len(LE_Warehouses) != len(top_warehouses):
#    raise SystemExit("Mismatch in number of warehouses")

In [253]:
# Merge based on Index
top_warehouses = pd.merge(
    top_warehouses,
    LE_Warehouses,
    how="left",
    on="Index",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [254]:
top_warehouses.drop(columns={'OrderLinesCount', 'Index'}, inplace=True, axis=1)

In [255]:
top_warehouses

,D365_Del_Loc,D365_Del_Loc_LE
0,ZA1-ZA014B,UG1-UG010B
1,ZA1-ZA020B,UG1-UG010B
2,ZA1-ZA508B,UG1-UG010B
3,ZA1-ZA557B,UG1-UG010B
4,ZA1-ZA537B,UG1-UG010B
...,...,...
82,ZA1-ZA543B,UG1-UG012B
83,ZA1-ZA025F,UG1-UG012B
84,ZA1-ZA042F,UG1-UG012B
85,ZA1-ZA200B,UG1-UG012B


### Merge Warehouses lookup table with SAP data

In [256]:
df5_1 = pd.merge(
    df5_W_C_P,
    top_warehouses,
    how="left",
    on="D365_Del_Loc",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [257]:
df5_W_C_P.info()

<class 'pandas.core.frame.DataFrame'>
Index: 630687 entries, 0 to 871830
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   ActivityType              630687 non-null  object
 1   Order Number              630687 non-null  int64 
 2   Material_Description      630687 non-null  object
 3   BaseDate                  630687 non-null  object
 4   BaseHour                  630687 non-null  int64 
 5   Cases                     630687 non-null  int64 
 6   Bill I                    627948 non-null  object
 7   Source Channel            630687 non-null  object
 8   Order Category            630687 non-null  object
 9   D365_ItemNo               630687 non-null  object
 10  Material No               630687 non-null  object
 11  MOD                       630687 non-null  int64 
 12  site_id                   630687 non-null  object
 13  WAREHOUSELOCATIONID       630687 non-null  object
 14  D365_Del_

In [258]:
df5_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630687 entries, 0 to 630686
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   ActivityType              630687 non-null  object
 1   Order Number              630687 non-null  int64 
 2   Material_Description      630687 non-null  object
 3   BaseDate                  630687 non-null  object
 4   BaseHour                  630687 non-null  int64 
 5   Cases                     630687 non-null  int64 
 6   Bill I                    627948 non-null  object
 7   Source Channel            630687 non-null  object
 8   Order Category            630687 non-null  object
 9   D365_ItemNo               630687 non-null  object
 10  Material No               630687 non-null  object
 11  MOD                       630687 non-null  int64 
 12  site_id                   630687 non-null  object
 13  WAREHOUSELOCATIONID       630687 non-null  object
 14  D365

### Merge Products lookup table with SAP data

In [259]:
df5_2 = pd.merge(
    df5_1,
    top_products,
    how="left",
    on="D365_ItemNo",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [260]:
df5_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630687 entries, 0 to 630686
Data columns (total 21 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   ActivityType              630687 non-null  object
 1   Order Number              630687 non-null  int64 
 2   Material_Description      630687 non-null  object
 3   BaseDate                  630687 non-null  object
 4   BaseHour                  630687 non-null  int64 
 5   Cases                     630687 non-null  int64 
 6   Bill I                    627948 non-null  object
 7   Source Channel            630687 non-null  object
 8   Order Category            630687 non-null  object
 9   D365_ItemNo               630687 non-null  object
 10  Material No               630687 non-null  object
 11  MOD                       630687 non-null  int64 
 12  site_id                   630687 non-null  object
 13  WAREHOUSELOCATIONID       630687 non-null  object
 14  D365

### Merge Customers lookup table with SAP data

In [261]:
df5_3 = pd.merge(
    df5_2,
    top_customers,
    how="left",
    on="D365_Cust_No",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [262]:
df5_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630687 entries, 0 to 630686
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   ActivityType              630687 non-null  object
 1   Order Number              630687 non-null  int64 
 2   Material_Description      630687 non-null  object
 3   BaseDate                  630687 non-null  object
 4   BaseHour                  630687 non-null  int64 
 5   Cases                     630687 non-null  int64 
 6   Bill I                    627948 non-null  object
 7   Source Channel            630687 non-null  object
 8   Order Category            630687 non-null  object
 9   D365_ItemNo               630687 non-null  object
 10  Material No               630687 non-null  object
 11  MOD                       630687 non-null  int64 
 12  site_id                   630687 non-null  object
 13  WAREHOUSELOCATIONID       630687 non-null  object
 14  D365

In [263]:
df5_3.drop(columns={'site_id', 'D365_Cust_No', 'Material_Description', 'D365_ItemNo', 'D365_Del_Loc'}, inplace=True, axis=1)

In [264]:
df5_3.rename(columns={'site_id_LE': 'site_id', 'D365_Del_Loc_LE': 'D365_Del_Loc', 'D365_Cust_No_LE': 'D365_Cust_No', 'Material_Description_LE': 'Material_Description', 'D365_ItemNo_LE': 'D365_ItemNo'}, inplace=True)

In [265]:
df5_3['BaseDate'] = pd.to_datetime(df5_3['BaseDate'])

In [266]:
# Drop duplicates based on Customer, D365_ItemNo, and BaseDate.
#df5_3 = df5_3.sort_values('Order Number').drop_duplicates(subset=['D365_Cust_No', 'D365_ItemNo', 'BaseDate'])
df5_3 = df5_3.sort_values('Order Number').drop_duplicates(subset=['Order Number', 'D365_ItemNo', 'BaseDate'])
#df5_3 = df5_3.sort_values('Order Number').drop_duplicates(subset=['ActivityType', 'D365_Cust_No', 'D365_ItemNo', 'BaseDate'])

In [267]:
path = './data/'+ LE + '/15Dec_D365_orders_' + LE + '.feather'
df5_3.to_feather(path)